# Color Mismatches in Stereoscopic Video: Real-World Dataset and Deep Correction Method

[Paper](https://arxiv.org/abs/2303.06657)
|
[Real-World Dataset](https://videoprocessing.ai/datasets/stereo-mismatch.html)
|
[WandB](https://wandb.ai/egorchistov/color-transfer)
|
[GitHub](https://github.com/egorchistov/color-transfer)

## Installation

Clone this repo and install dependencies:

In [ ]:
!git clone https://github.com/egorchistov/color-transfer.git
%cd color-transfer
!pip install -qr requirements.txt

First load test stereopair.

In [ ]:
from PIL import Image
from skimage.util import img_as_float
import torchvision.transforms.functional as F

gt = Image.open("graphics/0964_L.png")
reference = Image.open("graphics/0964_R.png")

target = F.adjust_hue(gt, hue_factor=0.5)

target, gt, reference = img_as_float(target), img_as_float(gt), img_as_float(reference) 

Then visualize methods results.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.color import rgb2gray

def plot_views(result):
    plt.figure(figsize=(16, 4))

    difference = np.abs(rgb2gray(gt - result)).clip(0, 1)
    difference /= difference.max()

    for i, (image, label) in enumerate(zip(
        [target, reference, result.clip(0, 1), difference],
        ["Target", "Reference", "Result", "Normalized Absolute Difference"]
    )):
        plt.subplot(1, 4, i + 1)
        plt.imshow(image, cmap="gray")
        plt.title(label)
        plt.xticks([])
        plt.yticks([])

    plt.tight_layout()

## Global Linear Methods

E. Reinhard, M. Ashikhmin, B. Gooch, and P. Shirley, “Color transfer
between images,” *IEEE Computer Graphics and Applications*, vol. 21,
pp. 34–41, 2001.

In [ ]:
from methods.linear import color_transfer_between_images as ct

plot_views(ct(target, reference))

X. Xiao and L. Ma, “Color transfer in correlated color space,” in
*Proceedings of the 2006 ACM International Conference on Virtual Reality
Continuum and Its Applications*, 2006, pp. 305–309.

In [ ]:
from methods.linear import color_transfer_in_correlated_color_space as ct_ccs

plot_views(ct_ccs(target, reference))

F. Pitié and A. Kokaram, “The linear monge-kantorovitch linear colour mapping for example-based colour transfer,” in *IET 4th European
Conference on Visual Media Production*. IEE, 2007, pp. 1–9.

In [ ]:
from methods.linear import monge_kantorovitch_color_transfer as mkct

plot_views(mkct(target, reference))

## Iterative Local Methods

F. Pitié, A. Kokaram, and R. Dahyot, “Automated colour grading using colour distribution transfer,” *Computer Vision and Image Understanding*,
vol. 107, no. 1–2, pp. 123–137, 2007.

In [ ]:
from methods.iterative import iterative_distribution_transfer as idt

plot_views(idt(target, reference))

F. Pitié, A. Kokaram, and R. Dahyot, “Automated colour grading using colour distribution transfer,” *Computer Vision and Image Understanding*,
vol. 107, no. 1–2, pp. 123–137, 2007.

In [ ]:
from methods.iterative import automated_color_grading as acg

plot_views(acg(target, reference))

## Neural Network-Based Methods

In [ ]:
import os
import torch
from wandb import Api
from kornia import image_to_tensor, tensor_to_image

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
api = Api()

S. Croci, C. Ozcinar, E. Zerman, R. Dudek, S. Knorr, and A. Smolic,
“Deep color mismatch correction in stereoscopic 3d images,” in *2021
IEEE International Conference on Image Processing (ICIP)*, 2021, pp.
1749–1753.

In [ ]:
from methods.dcmcs3di import DCMCS3DI

artifact = api.artifact("egorchistov/color-transfer/model-y1mq1usg:v0")
artifact_dir = artifact.download()
dcmcs3di = DCMCS3DI.load_from_checkpoint(os.path.join(artifact_dir, "model.ckpt"), map_location=device)
dcmcs3di.to(device)
dcmcs3di.eval()

@torch.no_grad()
def run_dcmcs3di(target, reference):
    target = image_to_tensor(target, keepdim=False).float().to(device)
    reference = image_to_tensor(reference, keepdim=False).float().to(device)

    height, width = target.shape[-2:]

    # Without downscaling the evaluation will result in OOM
    # In our comparison we used the GPUs with more memory and ran this method at full resolution
    target = torch.nn.functional.interpolate(target, scale_factor=0.75, mode="bicubic")
    reference = torch.nn.functional.interpolate(reference, scale_factor=0.75, mode="bicubic")

    result, _ = dcmcs3di(target, reference, inference=True)

    result = torch.nn.functional.interpolate(result, size=(height, width), mode="bicubic")

    return tensor_to_image(result)

plot_views(run_dcmcs3di(target, reference))

Our color-transfer method.

In [ ]:
from methods.dmsct import DMSCT

artifact = api.artifact("egorchistov/color-transfer/model-86n1v9bd:v0")
artifact_dir = artifact.download()
dmsct = DMSCT.load_from_checkpoint(os.path.join(artifact_dir, "model.ckpt"), map_location=device)
dmsct.to(device)
dmsct.eval()

@torch.no_grad()
def run_dmsct(target, reference):
    target = image_to_tensor(target, keepdim=False).float().to(device)
    reference = image_to_tensor(reference, keepdim=False).float().to(device)

    result = dmsct(target, reference)

    return tensor_to_image(result)

plot_views(run_dmsct(target, reference))